# Getting Started with the Temperature Control Laboratory

The following cells in this notebook demonstrate the use of the Temperature Control Lab. 

## TCLab Hardware

<img src='images/arduino_lab_kit.png' style="float: right;padding-left:30px;"></img>
The Temperature Control Laboratory consists of five components:

1. Arduino microcontroller board (Arduino Uno, Arduino Leonardo, or equivalents).

2. The Temperature Control Laboratory plug-in board (also known as a shield).

3. Five watt USB power supply.

4. 5.5mm to USB power supply cable.

5. USB 2.0 data cable. (w/mini-USB connector for Arduino Uno, or micro-USB cable for Arduino Leonardo.)

Before going further, be sure to complete the steps outlined under *Hardware setup* as described in TCLab [README](https://github.com/jckantor/TCLab/blob/master/README.rst). Mac OS users will need to install a serial driver available [here](https://github.com/adrianmihalko/ch340g-ch34g-ch34x-mac-os-x-driver). Normally the TCLab shield will already be mounted on the Arduino board, and the firmware driver will have been loaded on to the Arduino.

## TCLab Software
<img src='images/TCLabOverview.png' style="float: left;padding-right:30px;"></img>
**_Jupyter notebooks:_**
Examples, lessons, and student projects.

**_tclab:_**
A package to provide high-level access to sensors, heaters, a pseudo-realtime clock. The package includes `TCLab()` providing access to the device, `clock` for synchronizing with a real time clock, `Historian` for data logging and `Plotter` for realtime plotting.

**_sketch.ino:_**
Firmware for the intrisically safe operation of the Arduino board and shield.

**_Arduino:_**
Hardware platform for the Temperature Control Laboratory. Compatiable with Arduino Uno and Arduino Leonardo.

Before going further, be sure to install the `tclab` Python drivers. When used for the first time, the `tclab` package is installed with the command line

    pip install tclab

## Connecting to the Temperature Control Laboratory

TCLab() attempts to find a device connected to a serial port and returns a connection. An error is generated if no device is found.  The connection should be closed when no longer in use.

The following cell demonstrates this process, and uses the tclab `LED()` function to flash the LED on the Temperature Control Lab for a period of 10 seconds at a 100% brightness level. 

In [1]:
from tclab import TCLab

a = TCLab()
a.LED(100)
a.close()

Connecting to TCLab
TCLab Firmware Version 1.2.1 connected on port /dev/cu.wchusbserial1410
TCLab disconnected successfully.


### Using TCLab with Python's `with` statement

The Python `with` statement provides a convenient means of setting up and closing a connection to the Temperature Control Laboratory. In particular, the with statement establishes a context where a tclab instance is created, assigned to a variable, and automatically closed upon completion.

In [2]:
from tclab import TCLab

with TCLab() as a:
    a.LED(100)

Connecting to TCLab
TCLab Firmware Version 1.2.1 connected on port /dev/cu.wchusbserial1410
TCLab disconnected successfully.


The `with` statement is likely to be the most common way to connect the Temperature Control Laboratory for most uses.

## Reading Temperatures

Once a tclab instance is created and connected to a device, the temperature sensors on the temperature control lab can be acccessed with the attributes `.T1` and `.T2`.  For example, given an instance `a`, the temperatures are accessed as

    T1 = a.T1
    T2 = a.T2

Note that `a.T1` and `a.T2` are read-only properties. Any attempt to set them to a value will return a Python error.

In [3]:
from tclab import TCLab

with TCLab() as a:
    print("Temperature 1: {0:0.2f} °C".format(a.T1))
    print("Temperature 2: {0:0.2f} °C".format(a.T2))

Connecting to TCLab
TCLab Firmware Version 1.2.1 connected on port /dev/cu.wchusbserial1410
Temperature 1: 22.51 °C
Temperature 2: 22.51 °C
TCLab disconnected successfully.


## Setting Heaters

The heaters are controlled by functions`.Q1()` and `.Q2()` of a tclab instance. For example, both heaters can be set to 100% power with the functions

    a.Q1(100)
    a.Q2(100)

The device firmware limits the heaters to a range of 0 to 100%. The current value of attributes may be accessed via

    Q1 = a.Q1()
    Q2 = a.Q2()
    
Note that the retrieved values may be different due to the range-limiting enforced by the device firmware.

In [4]:
from tclab import TCLab
import time

with TCLab() as a:
    print("\nStarting Temperature 1: {0:0.2f} °C".format(a.T1),flush=True)
    print("Starting Temperature 2: {0:0.2f} °C".format(a.T2),flush=True)

    a.Q1(100)
    a.Q2(100)
    print("\nSet Heater 1:", a.Q1(), "%",flush=True)
    print("Set Heater 2:", a.Q2(), "%",flush=True)
    
    t_heat = 20
    print("\nHeat for", t_heat, "seconds")
    time.sleep(t_heat)

    print("\nTurn Heaters Off")
    a.Q1(0)
    a.Q2(0)
    print("\nSet Heater 1:", a.Q1(), "%",flush=True)
    print("Set Heater 2:", a.Q2(), "%",flush=True)
    
    print("\nFinal Temperature 1: {0:0.2f} °C".format(a.T1))
    print("Final Temperature 2: {0:0.2f} °C".format(a.T2))

Connecting to TCLab
TCLab Firmware Version 1.2.1 connected on port /dev/cu.wchusbserial1410

Starting Temperature 1: 22.51 °C
Starting Temperature 2: 22.83 °C

Set Heater 1: 100.0 %
Set Heater 2: 100.0 %

Heat for 20 seconds

Turn Heaters Off

Set Heater 1: 0.0 %
Set Heater 2: 0.0 %

Final Temperature 1: 24.44 °C
Final Temperature 2: 23.48 °C
TCLab disconnected successfully.


## Synchronizing with Real Time using `clock`

For convenience, the tclab module includes a function `clock` for synchronizing calculations with real time.  `clock(tperiod, tstep)` is a python iterator that generates a sequence of iterations over a period of `tperiod` seconds separated by `tstep` seconds. 

In [8]:
from tclab import TCLab, clock

tperiod = 10
tstep = 2
for t in clock(tperiod,tstep):
    print(t, "sec.")

0.0 sec.
2.0 sec.
4.0 sec.
6.0 sec.
8.0 sec.
9.99 sec.


## Using `clock` with TCLab

The following example shows how to integrate `clock` for use with TCLab.

In [10]:
from tclab import TCLab, clock

with TCLab() as a:   
    for t in clock(10):
        print(t,"sec., ", a.T1,"deg C, ", a.T2, "deg C")

Connecting to TCLab
TCLab Firmware Version 1.2.1 connected on port /dev/cu.wchusbserial1410
0.0 sec.,  23.15 deg C,  23.48 deg C
1.0 sec.,  22.83 deg C,  23.8 deg C
2.0 sec.,  22.83 deg C,  23.48 deg C
3.0 sec.,  22.83 deg C,  23.48 deg C
4.0 sec.,  22.83 deg C,  23.15 deg C
5.0 sec.,  22.83 deg C,  23.48 deg C
6.0 sec.,  23.15 deg C,  23.48 deg C
7.0 sec.,  23.15 deg C,  23.48 deg C
8.0 sec.,  23.15 deg C,  23.48 deg C
8.99 sec.,  22.83 deg C,  23.48 deg C
10.0 sec.,  22.83 deg C,  23.48 deg C
TCLab disconnected successfully.


## A Simple Thermostat

Let's build our first control system. This is a simple on/off thermostat intended to maintain temperature $T_1$ near a desired setpoint.

In [12]:
from tclab import TCLab, clock

SP = 37     # Setpoint

with TCLab() as a:   
    for t in clock(150, strict=True):
        # Measure temperature
        T1 = a.T1
        
        # Compute desired heater power
        if T1 < SP:
            Q1 = 100
        elif T1 >= SP:
            Q1 = 0
        
        # Set heater power
        a.Q1(Q1)
        
        # Record current state
        print(t,"sec.", "Q1 = ", Q1, "%", "T1 = ", T1, "deg C")

Connecting to TCLab
TCLab Firmware Version 1.2.1 connected on port /dev/cu.wchusbserial1410
0.0 sec. Q1 =  100 % T1 =  31.53 deg C
1.0 sec. Q1 =  100 % T1 =  31.86 deg C
2.0 sec. Q1 =  100 % T1 =  31.53 deg C
3.0 sec. Q1 =  100 % T1 =  31.86 deg C
4.0 sec. Q1 =  100 % T1 =  31.53 deg C
5.0 sec. Q1 =  100 % T1 =  31.53 deg C
6.0 sec. Q1 =  100 % T1 =  31.53 deg C
7.0 sec. Q1 =  100 % T1 =  31.53 deg C
8.0 sec. Q1 =  100 % T1 =  31.53 deg C
9.0 sec. Q1 =  100 % T1 =  31.86 deg C
10.0 sec. Q1 =  100 % T1 =  31.86 deg C
11.0 sec. Q1 =  100 % T1 =  31.86 deg C
12.0 sec. Q1 =  100 % T1 =  32.18 deg C
13.0 sec. Q1 =  100 % T1 =  32.5 deg C
14.0 sec. Q1 =  100 % T1 =  32.5 deg C
15.0 sec. Q1 =  100 % T1 =  32.82 deg C
16.0 sec. Q1 =  100 % T1 =  32.82 deg C
17.0 sec. Q1 =  100 % T1 =  33.14 deg C
18.0 sec. Q1 =  100 % T1 =  33.47 deg C
19.0 sec. Q1 =  100 % T1 =  33.79 deg C
20.0 sec. Q1 =  100 % T1 =  33.79 deg C
21.0 sec. Q1 =  100 % T1 =  34.11 deg C
22.0 sec. Q1 =  100 % T1 =  34.43 deg C
